In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import talib as ta
import datetime
import vectorbt as vbt
import optuna
import hiplot as hip
import warnings

warnings.filterwarnings("ignore")

In [ ]:
all_data = pd.read_csv('BTCUSDT_1h.csv')
all_data['open_time'] = pd.to_datetime(all_data['open_time'])

all_data = all_data.drop(['Unnamed: 0', 'close_time', 'ignore'], axis = 1)
all_data.columns = ['open_t', 'open', 'high', 'low', 'close', 'volume', 'USD_vol', 'trade_count', 'taker_buy_volume', 'taker_buy_USD_vol']

In [ ]:
# 取值到 2022 年 8 月 31 日 23:59:59 的数据作为 train_data
train_data = all_data[all_data['open_t'] <= '2022-08-31 23:59:59']

# 检查前几行数据
train_data.head()

In [ ]:
test_data = all_data[all_data['open_t'] >= '2022-08-31 23:59:59']

test_data.tail()

### 第一次優化（只使用 double ema + adx ） 
- MAX Calmer 

In [ ]:
import optuna
import vectorbt as vbt
import pandas as pd
import numpy as np
import talib as ta

# Objective function to optimize
def objective_calmar(trial):
    emafast_period = trial.suggest_int('emafast_period', 4, 72, 2)
    emaslow_period = trial.suggest_int('emaslow_period', 12, 96, 2)
    adx_threshold = trial.suggest_int('adx_threshold', 25, 50, 1)
    adx_period = trial.suggest_int('adx_period', 12, 48, 2)

    # Calculate the technical indicators on the full dataset
    train_data['EMAslow'] = ta.EMA(train_data['close'], timeperiod=emaslow_period)
    train_data['EMAfast'] = ta.EMA(train_data['close'], timeperiod=emafast_period)
    train_data['ADX'] = ta.ADX(train_data['high'], train_data['low'], train_data['close'], timeperiod=adx_period)

    # Check for NaN values in the indicators
 

    # Generate trading signals
    sig = pd.DataFrame(index=train_data.index)
    sig['long_entry'] = np.where((train_data['EMAfast'] > train_data['EMAslow']) & (train_data['ADX'] > adx_threshold), 1, 0)
    sig['long_exit'] = np.where(train_data['EMAfast'] < train_data['EMAslow'], 1, 0)
    sig['short_entry'] = np.where((train_data['EMAfast'] < train_data['EMAslow']) & (train_data['ADX'] > adx_threshold), 1, 0)
    sig['short_exit'] = np.where(train_data['EMAfast'] > train_data['EMAslow'], 1, 0)

    # Create a portfolio based on the signals
    pf = vbt.Portfolio.from_signals(
        train_data['close'],
        entries=sig['long_entry'],
        exits=sig['long_exit'],
        short_entries=sig['short_entry'],
        short_exits=sig['short_exit'],
        freq='h',
        fees=0.0004,
    )

    # Calculate the Calmar Ratio
    stats = pf.stats()
    calmar_ratio = stats['Calmar Ratio']
   
    return calmar_ratio

# Example dataset (replace this with your actual data)

# Set up and run the Optuna study
study = optuna.create_study(direction='maximize')
study.optimize(objective_calmar, n_trials=600)

# Print the best parameters and the corresponding Calmar Ratio
best_params = study.best_params
best_value = study.best_value

print(f"Best Parameters: {best_params}")
print(f"Best Calmar Ratio: {best_value}")


In [ ]:
hip.Experiment.from_optuna(study).display()

### Moving window OPT 
- to solve over fitting problem 
- trying to make 參數高原

In [ ]:
# import optuna
# import vectorbt as vbt
# import pandas as pd
# import numpy as np
# import talib as ta
# from sklearn.model_selection import TimeSeriesSplit

# # 定义时间序列数据分割函数
# def time_series_split(train_data, n_splits=5):
#     tscv = TimeSeriesSplit(n_splits=n_splits)
#     for train_index, test_index in tscv.split(train_data):
#         train, test = train_data.iloc[train_index], train_data.iloc[test_index]
#         yield train, test



# # 定义优化目标函数
# def objective_calmar(trial):
#     emafast_period = trial.suggest_int('emafast_period', 4, 72, 2)
#     emaslow_period = trial.suggest_int('emaslow_period', 12, 96, 2)
#     adx_threshold = trial.suggest_int('adx_threshold', 25, 52, 2)
#     adx_period = trial.suggest_int('adx_period', 12, 48, 2)

#     calmar_ratios = []

#     # 基本的策略评估
#     for train, test in time_series_split(train_data, n_splits=5):
#         train['EMAslow'] = ta.EMA(train['close'], timeperiod=emaslow_period)
#         train['EMAfast'] = ta.EMA(train['close'], timeperiod=emafast_period)
#         train['ADX'] = ta.ADX(train['high'], train['low'], train['close'], timeperiod=adx_period)

#         if train['EMAslow'].isna().sum() > 0 or train['EMAfast'].isna().sum() > 0 or train['ADX'].isna().sum() > 0:
#             return float('nan')

#         sig = pd.DataFrame(index=train.index)
#         sig['long_entry'] = np.where((train['EMAfast'] > train['EMAslow']) & (train['ADX'] > adx_threshold), 1, 0)
#         sig['long_exit'] = np.where(train['EMAfast'] < train['EMAslow'], 1, 0)
#         sig['short_entry'] = np.where((train['EMAfast'] < train['EMAslow']) & (train['ADX'] > adx_threshold), 1, 0)
#         sig['short_exit'] = np.where(train['EMAfast'] > train['EMAslow'], 1, 0)

#         pf = vbt.Portfolio.from_signals(
#             train['close'],
#             entries=sig['long_entry'],
#             exits=sig['long_exit'],
#             short_entries=sig['short_entry'],
#             short_exits=sig['short_exit'],
#             freq='h',
#             fees=0.0004,
#         )
        
#         stats = pf.stats()
#         calmar_ratio = stats['Calmar Ratio']
#         calmar_ratios.append(calmar_ratio)

#     # 计算当前参数的平均Calmar Ratio
#     base_calmar_ratio = np.mean(calmar_ratios)

#     # 邻域搜索，额外评估邻近的参数组合
#     neighborhood_ratios = []
#     for delta in [-4, 4]:  # 在这里定义邻域的范围
#         for d_emafast in [-delta, delta]:
#             for d_emaslow in [-delta, delta]:
#                 for d_adx in [-delta, delta]:
#                     for d_adx_period in [-delta, delta]:
#                         # 确保新的参数在合法范围内
#                         new_emafast = emafast_period + d_emafast
#                         new_emaslow = emaslow_period + d_emaslow
#                         new_adx = adx_threshold + d_adx
#                         new_adx_period = adx_period + d_adx_period
#                         if (4 <= new_emafast <= 72) and (12 <= new_emaslow <= 96) and (25 <= new_adx <= 52) and (12 <= new_adx_period <= 48):
#                             # 重新评估
#                             for train, test in time_series_split(train_data, n_splits=5):
#                                 train['EMAslow'] = ta.EMA(train['close'], timeperiod=new_emaslow)
#                                 train['EMAfast'] = ta.EMA(train['close'], timeperiod=new_emafast)
#                                 train['ADX'] = ta.ADX(train['high'], train['low'], train['close'], timeperiod=new_adx_period)

#                                 if train['EMAslow'].isna().sum() > 0 or train['EMAfast'].isna().sum() > 0 or train['ADX'].isna().sum() > 0:
#                                     continue

#                                 sig = pd.DataFrame(index=train.index)
#                                 sig['long_entry'] = np.where((train['EMAfast'] > train['EMAslow']) & (train['ADX'] > new_adx), 1, 0)
#                                 sig['long_exit'] = np.where(train['EMAfast'] < train['EMAslow'], 1, 0)
#                                 sig['short_entry'] = np.where((train['EMAfast'] < train['EMAslow']) & (train['ADX'] > new_adx), 1, 0)
#                                 sig['short_exit'] = np.where(train['EMAfast'] > train['EMAslow'], 1, 0)

#                                 pf = vbt.Portfolio.from_signals(
#                                     train['close'],
#                                     entries=sig['long_entry'],
#                                     exits=sig['long_exit'],
#                                     short_entries=sig['short_entry'],
#                                     short_exits=sig['short_exit'],
#                                     freq='h',
#                                     fees=0.0004,
#                                 )
                                
#                                 stats = pf.stats()
#                                 neighborhood_ratios.append(stats['Calmar Ratio'])

#     # 返回当前参数与邻域参数的平均值作为最终的优化目标
#     return (base_calmar_ratio + np.mean(neighborhood_ratios)) / 2 if neighborhood_ratios else base_calmar_ratio

# # 执行优化
# study = optuna.create_study(direction='maximize')
# study.optimize(objective_calmar, n_trials=600)

# # 最佳参数及其对应的 Calmar Ratio
# best_params = study.best_params
# print(f"Best Parameters: {best_params}")
# print("Best value (Calmar Ratio):", study.best_value)


In [ ]:
# Retrieve best parameters from the Optuna study
EMAfast_p = study.best_params['emafast_period']
EMAslow_p = study.best_params['emaslow_period']
adx_p = study.best_params['adx_period']
ADX_threshold_p = study.best_params['adx_threshold']

# Calculate technical indicators using the best parameters
train_data['EMAslow'] = ta.EMA(train_data['close'], timeperiod=EMAslow_p)
train_data['EMAfast'] = ta.EMA(train_data['close'], timeperiod=EMAfast_p)
train_data['ADX'] = ta.ADX(train_data['high'], train_data['low'], train_data['close'], timeperiod=adx_p)

# Generate entry and exit signals
sig = pd.DataFrame(index=train_data.index)

# Long entry signal
sig['long_entry'] = np.where(
    (train_data['EMAfast'] > train_data['EMAslow']) & 
    (train_data['ADX'] > ADX_threshold_p), 
    1, 
    0
)

# Long exit signal
sig['long_exit'] = np.where(
    train_data['EMAfast'] < train_data['EMAslow'], 
    1, 
    0
)

# Short entry signal
sig['short_entry'] = np.where(
    (train_data['EMAfast'] < train_data['EMAslow']) & 
    (train_data['ADX'] > ADX_threshold_p), 
    1, 
    0
)

# Short exit signal
sig['short_exit'] = np.where(
    train_data['EMAfast'] > train_data['EMAslow'], 
    1, 
    0
)

# Create portfolio and calculate statistics
pft = vbt.Portfolio.from_signals(
    train_data['close'],
    entries=sig['long_entry'],
    exits=sig['long_exit'],
    short_entries=sig['short_entry'],
    short_exits=sig['short_exit'],
    freq='h',
    fees=0.0004
)

# Output portfolio statistics
pft_stats = pft.stats()
print(pft_stats)


In [ ]:
initial_value = 1  
total_return_percent = pft.total_return() 

# 計算最終資產價值
end_value = initial_value * (1 + total_return_percent)

# 計算投資年數
investment_years = 970 / 365  # 根據總的投資天數計算年數

# 根據複利公式計算年化報酬率
annualized_return = (end_value / initial_value) ** (1 / investment_years) - 1

# Calculate annualized volatility (based on daily returns)
hourly_returns = pft.returns()
hourly_returns.index = pd.date_range(start='2020-01-01', periods=len(hourly_returns), freq='H')

daily_returns = (1 + hourly_returns).resample('D').prod() - 1

annualized_volatility = daily_returns.std() * (365 ** 0.5)

# Calculate annualized Sharpe ratio (assuming risk-free rate is 0)
annualized_sharpe = annualized_return / annualized_volatility

# Calculate Maximum Drawdown (MDD) and risk-reward ratio (Calmar Ratio)
mdd = pft.max_drawdown()
risk_reward_ratio = -(pft.total_return()) / (mdd)

# Print all calculated statistics
print(f"Annualized Return: {annualized_return:.2%}")
print(f"Annualized Volatility: {annualized_volatility:.2%}")
print(f"Annualized Sharpe Ratio: {annualized_sharpe:.2f}")
print(f"Max Drawdown: {mdd:.2%}")
print(f"Risk-Reward Ratio (Calmar Ratio): {risk_reward_ratio:.2f}")




In [ ]:
# Retrieve best parameters from the Optuna study
EMAfast_p = study.best_params['emafast_period']
EMAslow_p = study.best_params['emaslow_period']
adx_p = study.best_params['adx_period']
adx_threshold_p = study.best_params['adx_threshold']

# Calculate technical indicators using the best parameters
test_data['EMAslow'] = ta.EMA(test_data['close'], timeperiod=EMAslow_p)
test_data['EMAfast'] = ta.EMA(test_data['close'], timeperiod=EMAfast_p)
test_data['ADX'] = ta.ADX(test_data['high'], test_data['low'], test_data['close'], timeperiod=adx_p)

# Generate entry and exit signals
sig = pd.DataFrame(index=test_data.index)

# Long entry signal
sig['long_entry'] = np.where(
    (test_data['EMAfast'] > test_data['EMAslow']) & 
    (test_data['ADX'] > adx_threshold_p), 
    1, 
    0
)

# Long exit signal
sig['long_exit'] = np.where(
    test_data['EMAfast'] < test_data['EMAslow'], 
    1, 
    0
)

# Short entry signal
sig['short_entry'] = np.where(
    (test_data['EMAfast'] < test_data['EMAslow']) & 
    (test_data['ADX'] > adx_threshold_p), 
    1, 
    0
)

# Short exit signal
sig['short_exit'] = np.where(
    test_data['EMAfast'] > test_data['EMAslow'], 
    1, 
    0
)

# Create portfolio and calculate statistics
pft = vbt.Portfolio.from_signals(
    test_data['close'],
    entries=sig['long_entry'],
    exits=sig['long_exit'],
    short_entries=sig['short_entry'],
    short_exits=sig['short_exit'],
    freq='h',
    fees=0.0004
)

# Output portfolio statistics
pft_stats = pft.stats()
print(pft_stats)


In [ ]:
initial_value = 1  
total_return_percent = pft.total_return() 

# 計算最終資產價值
end_value = initial_value * (1 + total_return_percent)

# 計算投資年數
investment_years = 669 / 365  # 根據總的投資天數計算年數

# 根據複利公式計算年化報酬率
annualized_return = (end_value / initial_value) ** (1 / investment_years) - 1

# Calculate annualized volatility (based on daily returns)
hourly_returns = pft.returns()

hourly_returns.index = pd.date_range(start='2022-09-01', periods=len(hourly_returns), freq='H')

daily_returns = (1 + hourly_returns).resample('D').prod() - 1

annualized_volatility = daily_returns.std() * (365 ** 0.5)

# Calculate annualized Sharpe ratio (assuming risk-free rate is 0)
annualized_sharpe = annualized_return / annualized_volatility

# Calculate Maximum Drawdown (MDD) and risk-reward ratio (Calmar Ratio)
mdd = pft.max_drawdown()
risk_reward_ratio = -(pft.total_return()) / (mdd)

# Print all calculated statistics
print(f"Annualized Return: {annualized_return:.2%}")
print(f"Annualized Volatility: {annualized_volatility:.2%}")
print(f"Annualized Sharpe Ratio: {annualized_sharpe:.2f}")
print(f"Max Drawdown: {mdd:.2%}")
print(f"Risk-Reward Ratio (Calmar Ratio): {risk_reward_ratio:.2f}")




In [ ]:
# Retrieve best parameters from the Optuna study
EMAfast_p = study.best_params['emafast_period']
EMAslow_p = study.best_params['emaslow_period']
ADX_threshold_p = study.best_params['adx_threshold']
adx_p = study.best_params['adx_period']


# Calculate technical indicators using the best parameters
all_data['EMAslow'] = ta.EMA(all_data['close'], timeperiod=EMAslow_p)
all_data['EMAfast'] = ta.EMA(all_data['close'], timeperiod=EMAfast_p)
all_data['ADX'] = ta.ADX(all_data['high'], all_data['low'], all_data['close'], timeperiod=adx_p)

# Generate entry and exit signals
sig = pd.DataFrame(index=all_data.index)

# Long entry signal
sig['long_entry'] = np.where(
    (all_data['EMAfast'] > all_data['EMAslow']) & 
    (all_data['ADX'] > ADX_threshold_p), 
    1, 
    0
)

# Long exit signal
sig['long_exit'] = np.where(
    all_data['EMAfast'] < all_data['EMAslow'], 
    1, 
    0
)

# Short entry signal
sig['short_entry'] = np.where(
    (all_data['EMAfast'] < all_data['EMAslow']) & 
    (all_data['ADX'] > ADX_threshold_p), 
    1, 
    0
)

# Short exit signal
sig['short_exit'] = np.where(
    all_data['EMAfast'] > all_data['EMAslow'], 
    1, 
    0
)

# Create portfolio and calculate statistics
pft = vbt.Portfolio.from_signals(
    all_data['close'],
    entries=sig['long_entry'],
    exits=sig['long_exit'],
    short_entries=sig['short_entry'],
    short_exits=sig['short_exit'],
    freq='h',
    fees=0.0004
)

# Output portfolio statistics
pft_stats = pft.stats()
print(pft_stats)


In [ ]:
initial_value = 1  
total_return_percent = pft.total_return() 

# 計算最終資產價值
end_value = initial_value * (1 + total_return_percent)

# 計算投資年數
investment_years = 1639 / 365  # 根據總的投資天數計算年數

# 根據複利公式計算年化報酬率
annualized_return = (end_value / initial_value) ** (1 / investment_years) - 1

# Calculate annualized volatility (based on daily returns)
hourly_returns = pft.returns()

hourly_returns.index = pd.date_range(start='2020-01-01', periods=len(hourly_returns), freq='H')

daily_returns = (1 + hourly_returns).resample('D').prod() - 1

annualized_volatility = daily_returns.std() * (365 ** 0.5)

# Calculate annualized Sharpe ratio (assuming risk-free rate is 0)
annualized_sharpe = annualized_return / annualized_volatility

# Calculate Maximum Drawdown (MDD) and risk-reward ratio (Calmar Ratio)
mdd = pft.max_drawdown()
risk_reward_ratio = -(pft.total_return()) / (mdd)

# Print all calculated statistics
print(f"Annualized Return: {annualized_return:.2%}")
print(f"Annualized Volatility: {annualized_volatility:.2%}")
print(f"Annualized Sharpe Ratio: {annualized_sharpe:.2f}")
print(f"Max Drawdown: {mdd:.2%}")
print(f"Risk-Reward Ratio (Calmar Ratio): {risk_reward_ratio:.2f}")




In [ ]:
fig = pft.plot(
    subplots=['cum_returns', 'orders', 'trade_pnl', 'drawdowns'], 
    autosize=False, 
    width=1200, 
    height=800  # Increased height to accommodate more subplots
)
fig.show()


### 改進 
- setting rolling window of volatility and pct_rank to optimize 
- Optimize only once 

In [ ]:
import optuna
import vectorbt as vbt
import numpy as np
import pandas as pd
import talib as ta  

# 定義優化目標函數，最大化 Calmar Ratio
def objective_calmar(trial):
    # 定義要優化的參數範圍
    emafast_period = trial.suggest_int('emafast_period', 4, 40, 2)
    emaslow_period = trial.suggest_int('emaslow_period', 20, 200, 4)
    adx_threshold = trial.suggest_int('adx_threshold', 40, 80, 2)
    adx_period = trial.suggest_int('adx_period', 10, 30, 2)
    
    # 優化波動率相關的窗口參數
    volatility_window = trial.suggest_int('volatility_window', 10, 24, 2)
    pctrank_window = trial.suggest_int('pctrank_window', 24, 48, 2)

    # 計算技術指標
    train_data['EMAslow'] = ta.EMA(train_data['close'], timeperiod=emaslow_period)
    train_data['EMAfast'] = ta.EMA(train_data['close'], timeperiod=emafast_period)
    train_data['ADX'] = ta.ADX(train_data['high'], train_data['low'], train_data['close'], timeperiod=adx_period)

    # 計算對數收益率和波動率
    train_data['log_return'] = np.log(train_data['close'] / train_data['close'].shift(1)) 
    train_data['volatility'] = train_data['log_return'].rolling(window=volatility_window).std()
    train_data['volatility_pctrank'] = train_data['volatility'].rolling(window=pctrank_window).apply(lambda x: pd.Series(x).rank(pct=True).iloc[-1], raw=True)

    # 生成交易信號
    sig = pd.DataFrame(index=train_data.index)
    sig['long_entry'] = np.where(
        (train_data['EMAfast'] > train_data['EMAslow']) & 
        (train_data['ADX'] > adx_threshold) &
        (train_data['volatility_pctrank'] >= 0.25) & (train_data['volatility_pctrank'] <= 0.75), 1, 0
    )
    sig['long_exit'] = np.where(train_data['EMAfast'] < train_data['EMAslow'], 1, 0)
    sig['short_entry'] = np.where(
        (train_data['EMAfast'] < train_data['EMAslow']) & 
        (train_data['ADX'] > adx_threshold) &
        (train_data['volatility_pctrank'] >= 0.25) & (train_data['volatility_pctrank'] <= 0.75), 1, 0
    )
    sig['short_exit'] = np.where(train_data['EMAfast'] > train_data['EMAslow'], 1, 0)

    # 創建投資組合並計算統計數據
    pf = vbt.Portfolio.from_signals(
        train_data['close'],
        entries=sig['long_entry'],
        exits=sig['long_exit'],
        short_entries=sig['short_entry'],
        short_exits=sig['short_exit'],
        freq='h',
        fees=0.0004,
    )
    
    stats = pf.stats()

    # 目標：最大化 Calmar Ratio
    calmar_ratio = stats['Calmar Ratio']
    
    return calmar_ratio

# 執行優化
study = optuna.create_study(direction='maximize')
study.optimize(objective_calmar, n_trials=400)

# 最佳參數及其對應的 Calmar Ratio
best_params = study.best_params
print(f"Best Parameters: {best_params}")
print("Best value (Calmar Ratio):", study.best_value)


In [ ]:
import optuna
import vectorbt as vbt
import numpy as np
import pandas as pd
import talib as ta  # 注意这里使用的是 TA-Lib 而非 ta 库

# 定义优化目标函数，最大化 Calmar Ratio
def objective_calmar(trial):
    # 定义要优化的参数范围
    emafast_period = trial.suggest_int('emafast_period', 5, 30, 1)
    emaslow_period = trial.suggest_int('emaslow_period', 50, 150, 2)
    adx_threshold = trial.suggest_int('adx_threshold', 20, 50, 1)
    adx_period = trial.suggest_int('adx_period', 10, 25, 1)
    
    # 优化波动率相关的窗口参数
    volatility_window = trial.suggest_int('volatility_window', 14, 50, 1)
    pctrank_window = trial.suggest_int('pctrank_window', 30, 100, 5)

    # 计算技术指标
    train_data['EMAslow'] = ta.EMA(train_data['close'], timeperiod=emaslow_period)
    train_data['EMAfast'] = ta.EMA(train_data['close'], timeperiod=emafast_period)
    train_data['ADX'] = ta.ADX(train_data['high'], train_data['low'], train_data['close'], timeperiod=adx_period)

    # 计算对数收益率和波动率
    train_data['log_return'] = np.log(train_data['close'] / train_data['close'].shift(1))
    train_data['volatility'] = train_data['log_return'].rolling(window=volatility_window).std()
    train_data['volatility_pctrank'] = train_data['volatility'].rolling(window=pctrank_window).apply(lambda x: pd.Series(x).rank(pct=True).iloc[-1], raw=True)

    # 生成交易信号
    sig = pd.DataFrame(index=train_data.index)
    sig['long_entry'] = np.where(
        (train_data['EMAfast'] > train_data['EMAslow']) & 
        (train_data['ADX'] > adx_threshold) &
        (train_data['volatility_pctrank'] >= 0.25) & (train_data['volatility_pctrank'] <= 0.75), 1, 0
    )
    sig['long_exit'] = np.where(train_data['EMAfast'] < train_data['EMAslow'], 1, 0)
    sig['short_entry'] = np.where(
        (train_data['EMAfast'] < train_data['EMAslow']) & 
        (train_data['ADX'] > adx_threshold) &
        (train_data['volatility_pctrank'] >= 0.25) & (train_data['volatility_pctrank'] <= 0.75), 1, 0
    )
    sig['short_exit'] = np.where(train_data['EMAfast'] > train_data['EMAslow'], 1, 0)

    # 创建投资组合并计算统计数据
    pf = vbt.Portfolio.from_signals(
        train_data['close'],
        entries=sig['long_entry'],
        exits=sig['long_exit'],
        short_entries=sig['short_entry'],
        short_exits=sig['short_exit'],
        freq='h',
        fees=0.0004,
    )
    
    stats = pf.stats()

    # 目标：最大化 Calmar Ratio
    calmar_ratio = stats['Calmar Ratio']
    
    return calmar_ratio

# 执行优化
study = optuna.create_study(direction='maximize')
study.optimize(objective_calmar, n_trials=400)

# 最佳参数及其对应的 Calmar Ratio
best_params = study.best_params
print(f"Best Parameters: {best_params}")
print("Best value (Calmar Ratio):", study.best_value)


In [ ]:
hip.Experiment.from_optuna(study).display()

In [ ]:
# Retrieve best parameters from the Optuna study
EMAfast_p = study.best_params['emafast_period']
EMAslow_p = study.best_params['emaslow_period']
ADX_threshold_p = study.best_params['adx_threshold']
ADX_period_p = study.best_params['adx_period']
volatility_window_p = study.best_params['volatility_window']
pctrank_window_p = study.best_params['pctrank_window']

# Calculate technical indicators using the best parameters
train_data['EMAslow'] = ta.EMA(train_data['close'], timeperiod=EMAslow_p)
train_data['EMAfast'] = ta.EMA(train_data['close'], timeperiod=EMAfast_p)
train_data['ADX'] = ta.ADX(train_data['high'], train_data['low'], train_data['close'], timeperiod=ADX_period_p)

# Calculate log return and volatility using the optimized window
train_data['log_return'] = np.log(train_data['close'] / train_data['close'].shift(1))
train_data['volatility'] = train_data['log_return'].rolling(window=volatility_window_p).std()
train_data['volatility_pctrank'] = train_data['volatility'].rolling(window=pctrank_window_p).apply(
    lambda x: pd.Series(x).rank(pct=True).iloc[-1], raw=True)

# Generate entry and exit signals
sig = pd.DataFrame(index=train_data.index)
sig['long_entry'] = np.where(
    (train_data['EMAfast'] > train_data['EMAslow']) & 
    (train_data['ADX'] > ADX_threshold_p) &
    (train_data['volatility_pctrank'] >= 0.25) & (train_data['volatility_pctrank'] <= 0.75), 
    1, 0
)
sig['long_exit'] = np.where(train_data['EMAfast'] < train_data['EMAslow'], 1, 0)
sig['short_entry'] = np.where(
    (train_data['EMAfast'] < train_data['EMAslow']) & 
    (train_data['ADX'] > ADX_threshold_p) &
    (train_data['volatility_pctrank'] >= 0.25) & (train_data['volatility_pctrank'] <= 0.75), 
    1, 0
)
sig['short_exit'] = np.where(train_data['EMAfast'] > train_data['EMAslow'], 1, 0)

# Create portfolio and calculate statistics
pft = vbt.Portfolio.from_signals(
    train_data['close'],
    entries=sig['long_entry'],
    exits=sig['long_exit'],
    short_entries=sig['short_entry'],
    short_exits=sig['short_exit'],
    freq='h',
    fees=0.0004
)

# Output portfolio statistics
pft_stats = pft.stats()
print(pft_stats)


In [ ]:
initial_value = 1  
total_return_percent = pft.total_return() 

# 計算最終資產價值
end_value = initial_value * (1 + total_return_percent)
# 計算投資年數
investment_years = 970 / 365  # 根據總的投資天數計算年數

# 根據複利公式計算年化報酬率
annualized_return = (end_value / initial_value) ** (1 / investment_years) - 1

# Calculate annualized volatility (based on daily returns)
hourly_returns = pft.returns()

hourly_returns.index = pd.date_range(start='2020-01-01', periods=len(hourly_returns), freq='H')

daily_returns = (1 + hourly_returns).resample('D').prod() - 1

annualized_volatility = daily_returns.std() * (365 ** 0.5)

# Calculate annualized Sharpe ratio (assuming risk-free rate is 0)
annualized_sharpe = annualized_return / annualized_volatility

# Calculate Maximum Drawdown (MDD) and risk-reward ratio (Calmar Ratio)
mdd = pft.max_drawdown()
risk_reward_ratio = -(pft.total_return()) / (mdd)

# Print all calculated statistics
print(f"Annualized Return: {annualized_return:.2%}")
print(f"Annualized Volatility: {annualized_volatility:.2%}")
print(f"Annualized Sharpe Ratio: {annualized_sharpe:.2f}")
print(f"Max Drawdown: {mdd:.2%}")
print(f"Risk-Reward Ratio (Calmar Ratio): {risk_reward_ratio:.2f}")


In [ ]:
# Retrieve best parameters from the Optuna study
EMAfast_p = study.best_params['emafast_period']
EMAslow_p = study.best_params['emaslow_period']
ADX_threshold_p = study.best_params['adx_threshold']
ADX_period_p = study.best_params['adx_period']
volatility_window_p = study.best_params['volatility_window']
pctrank_window_p = study.best_params['pctrank_window']

# Calculate technical indicators using the best parameters
test_data['EMAslow'] = ta.EMA(test_data['close'], timeperiod=EMAslow_p)
test_data['EMAfast'] = ta.EMA(test_data['close'], timeperiod=EMAfast_p)
test_data['ADX'] = ta.ADX(test_data['high'], test_data['low'], test_data['close'], timeperiod=ADX_period_p)

# Calculate log return and volatility using the optimized window
test_data['log_return'] = np.log(test_data['close'] / test_data['close'].shift(1))
test_data['volatility'] = test_data['log_return'].rolling(window=volatility_window_p).std()
test_data['volatility_pctrank'] = test_data['volatility'].rolling(window=pctrank_window_p).apply(
    lambda x: pd.Series(x).rank(pct=True).iloc[-1], raw=True)

# Generate entry and exit signals
sig = pd.DataFrame(index=test_data.index)
sig['long_entry'] = np.where(
    (test_data['EMAfast'] > test_data['EMAslow']) & 
    (test_data['ADX'] > ADX_threshold_p) &
    (test_data['volatility_pctrank'] >= 0.25) & (test_data['volatility_pctrank'] <= 0.75), 
    1, 0
)
sig['long_exit'] = np.where(test_data['EMAfast'] < test_data['EMAslow'], 1, 0)
sig['short_entry'] = np.where(
    (test_data['EMAfast'] < test_data['EMAslow']) & 
    (test_data['ADX'] > ADX_threshold_p) &
    (test_data['volatility_pctrank'] >= 0.25) & (test_data['volatility_pctrank'] <= 0.75), 
    1, 0
)
sig['short_exit'] = np.where(test_data['EMAfast'] > test_data['EMAslow'], 1, 0)

# Create portfolio and calculate statistics
pft = vbt.Portfolio.from_signals(
    test_data['close'],
    entries=sig['long_entry'],
    exits=sig['long_exit'],
    short_entries=sig['short_entry'],
    short_exits=sig['short_exit'],
    freq='h',
    fees=0.0004
)

# Output portfolio statistics
pft_stats = pft.stats()
print(pft_stats)


In [ ]:
initial_value = 1  
total_return_percent = pft.total_return() 

# 計算最終資產價值
end_value = initial_value * (1 + total_return_percent)

# 計算投資年數
investment_years = 669 / 365  # 根據總的投資天數計算年數

# 根據複利公式計算年化報酬率
annualized_return = (end_value / initial_value) ** (1 / investment_years) - 1

# Calculate annualized volatility (based on daily returns)
hourly_returns = pft.returns()

hourly_returns.index = pd.date_range(start='2022-09-01', periods=len(hourly_returns), freq='H')

daily_returns = (1 + hourly_returns).resample('D').prod() - 1

annualized_volatility = daily_returns.std() * (365 ** 0.5)

# Calculate annualized Sharpe ratio (assuming risk-free rate is 0)
annualized_sharpe = annualized_return / annualized_volatility

# Calculate Maximum Drawdown (MDD) and risk-reward ratio (Calmar Ratio)
mdd = pft.max_drawdown()
risk_reward_ratio = -(pft.total_return()) / (mdd)

# Print all calculated statistics
print(f"Annualized Return: {annualized_return:.2%}")
print(f"Annualized Volatility: {annualized_volatility:.2%}")
print(f"Annualized Sharpe Ratio: {annualized_sharpe:.2f}")
print(f"Max Drawdown: {mdd:.2%}")
print(f"Risk-Reward Ratio (Calmar Ratio): {risk_reward_ratio:.2f}")




In [ ]:
# Retrieve best parameters from the Optuna study
EMAfast_p = study.best_params['emafast_period']
EMAslow_p = study.best_params['emaslow_period']
ADX_threshold_p = study.best_params['adx_threshold']
ADX_period_p = study.best_params['adx_period']
volatility_window_p = study.best_params['volatility_window']
pctrank_window_p = study.best_params['pctrank_window']

# Calculate technical indicators using the best parameters
all_data['EMAslow'] = ta.EMA(all_data['close'], timeperiod=EMAslow_p)
all_data['EMAfast'] = ta.EMA(all_data['close'], timeperiod=EMAfast_p)
all_data['ADX'] = ta.ADX(all_data['high'], all_data['low'], all_data['close'], timeperiod=ADX_period_p)

# Calculate log return and volatility using the optimized window
all_data['log_return'] = np.log(all_data['close'] / all_data['close'].shift(1))
all_data['volatility'] = all_data['log_return'].rolling(window=volatility_window_p).std()
all_data['volatility_pctrank'] = all_data['volatility'].rolling(window=pctrank_window_p).apply(
    lambda x: pd.Series(x).rank(pct=True).iloc[-1], raw=True)

# Generate entry and exit signals
sig = pd.DataFrame(index=all_data.index)
sig['long_entry'] = np.where(
    (all_data['EMAfast'] > all_data['EMAslow']) & 
    (all_data['ADX'] > ADX_threshold_p) &
    (all_data['volatility_pctrank'] >= 0.25) & (all_data['volatility_pctrank'] <= 0.75), 
    1, 0
)
sig['long_exit'] = np.where(all_data['EMAfast'] < all_data['EMAslow'], 1, 0)
sig['short_entry'] = np.where(
    (all_data['EMAfast'] < all_data['EMAslow']) & 
    (all_data['ADX'] > ADX_threshold_p) &
    (all_data['volatility_pctrank'] >= 0.25) & (all_data['volatility_pctrank'] <= 0.75), 
    1, 0
)
sig['short_exit'] = np.where(all_data['EMAfast'] > all_data['EMAslow'], 1, 0)

# Create portfolio and calculate statistics
pft = vbt.Portfolio.from_signals(
    all_data['close'],
    entries=sig['long_entry'],
    exits=sig['long_exit'],
    short_entries=sig['short_entry'],
    short_exits=sig['short_exit'],
    freq='h',
    fees=0.0004
)

# Output portfolio statistics
pft_stats = pft.stats()
print(pft_stats)


In [ ]:
fig = pft.plot(
    subplots=['cum_returns', 'orders', 'trade_pnl', 'drawdowns'], 
    autosize=False, 
    width=1000, 
    height=600  # Increased height to accommodate more subplots
)
fig.show()


In [ ]:
initial_value = 1  
total_return_percent = pft.total_return() 

# 計算最終資產價值
end_value = initial_value * (1 + total_return_percent)

# 計算投資年數
investment_years = 1639 / 365  # 根據總的投資天數計算年數

# 根據複利公式計算年化報酬率
annualized_return = (end_value / initial_value) ** (1 / investment_years) - 1

# Calculate annualized volatility (based on daily returns)
hourly_returns = pft.returns()

hourly_returns.index = pd.date_range(start='2020-01-01', periods=len(hourly_returns), freq='H')

daily_returns = (1 + hourly_returns).resample('D').prod() - 1

annualized_volatility = daily_returns.std() * (365 ** 0.5)

# Calculate annualized Sharpe ratio (assuming risk-free rate is 0)
annualized_sharpe = annualized_return / annualized_volatility

# Calculate Maximum Drawdown (MDD) and risk-reward ratio (Calmar Ratio)
mdd = pft.max_drawdown()
risk_reward_ratio = -(pft.total_return()) / (mdd)

# Print all calculated statistics
print(f"Annualized Return: {annualized_return:.2%}")
print(f"Annualized Volatility: {annualized_volatility:.2%}")
print(f"Annualized Sharpe Ratio: {annualized_sharpe:.2f}")
print(f"Max Drawdown: {mdd:.2%}")
print(f"Risk-Reward Ratio (Calmar Ratio): {risk_reward_ratio:.2f}")


